In [1]:

from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components import CallChannel, ComponentStarter

from components.two_wheels import TwoWheelsV3
from components.logger import LoggerComponent
from components.controller.bluetooth_controller import BlueToothCarControlSPP_V2
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2b
from components.gyroscope.gyroscope import AngularSpeedControlV3
from components.camera import Picamera2V2
from data_collection.data_collection import LoggerSet
from components import PitchAngularVelocityController, get_switches
from multiprocessing import Manager
import numpy as np
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)

# from components import numpy_sample_setup
# r, w = numpy_sample_setup(['d', 'd'], [0, 0])

%load_ext autoreload
%autoreload 2

In [2]:
loggerset = LoggerSet('../log/testsession'+str(datetime.datetime.now()), overwrite_ok=False)
#loggerset = LoggerSet('../log/session'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

In [3]:

logger_process = ComponentStarter(
    LoggerComponent, 
    manager, 
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_intervals={'step': 1/100},
)

two_wheel_process = ComponentStarter(
    TwoWheelsV3, 
    manager, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = TwoWheelsV3.entry

)

angular_speed_control_process = ComponentStarter(
    AngularSpeedControlV3, 
    manager, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_intervals={'step': 1/100},
    instantiator = AngularSpeedControlV3.entry
)

bluetooth_control_process = ComponentStarter(
    BlueToothCarControlSPP_V2, 
    manager, 
    init_kwargs={},
    loop_intervals={'step': 1/100},
)

bt_ser_out, bt_ser_process_man = start_bluetooth_server_v2b(manager)


picamera_process = ComponentStarter(
    Picamera2V2, 
    manager, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_intervals={'step': 1/30},
    sample_setup_kwargs=dict(default_values=[np.zeros((64, 114, 3), dtype=np.uint8)])
)

pitch_process = ComponentStarter(
    PitchAngularVelocityController,
    manager, 
    init_kwargs=dict(speed=100),
    loop_intervals={'step': 1/30}, 
    instantiator=PitchAngularVelocityController.entry
    

)


In [4]:
two_wheel_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

angular_speed_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)


bluetooth_control_process.register_outgoing_rpc(
    dict(log=logger_process.incoming_rpcs['log'])
)

picamera_process.register_outgoing_rpc(
    dict(
        log=logger_process.incoming_rpcs['log'],
        setup_video_saver=logger_process.incoming_rpcs['setup_video_saver'],
        save_video_frame=logger_process.incoming_rpcs['save_video_frame']
        )
)

def get_pitch_drive_switch():
    return bool(bt_ser_out[0]().get('start'))

to_imu_controller = get_switches(pitch_process.outgoing_samples, bluetooth_control_process.outgoing_samples, get_pitch_drive_switch)


two_wheel_process.register_incoming_samples(
    angular_speed_control_process.outgoing_sample_readers
)

angular_speed_control_process.register_incoming_samples(
    to_imu_controller
)


bluetooth_control_process.register_incoming_samples(
    bt_ser_out
)


In [5]:
bt_ser_process_man.start()
bluetooth_control_process.start()
logger_process.start()
two_wheel_process.start()
angular_speed_control_process.start()
picamera_process.start()
pitch_process.start()


Making discoverable...
Starting Serial Port Profile...


/home/kawa/projects/car1/src/components/syncronisation/__init__.py:333: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")


/home/kawa/projects/car1/src/components/two_wheels.py:17: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:73: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:74: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/syncronisation/__init__.py:333: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
[0:32:09.679180523] [9175]  INFO Camera camera_manager.cpp:313 libcamera v0.3.0+65-6ddd79b5-dirty (2024-06-24T11:13:49+10:00)
[0:32:09.743577656] [9268

using device: USB PnP Sound Device, USB Audio


/home/kawa/projects/car1/src/components/syncronisation/__init__.py:333: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
/home/kawa/projects/car1/src/components/syncronisation/__init__.py:333: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
/home/kawa/projects/car1/src/components/microphone/ahhhhh.py:181: RuntimeWarning: divide by zero encountered in divide
  return 1/freqs[:len(freqs)//2], pitch_power[:len(freqs)//2]


NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 95)
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: BlueToothCarControlSPP_V2
saving: TwoWheelsV3
saving: Picamera2V2
saving: AngularSpeedControlV3
saving:

In [27]:
bt_ser_process_man.kill()
bluetooth_control_process.process_starter.kill()
logger_process.process_starter.kill()
two_wheel_process.process_starter.kill()
angular_speed_control_process.process_starter.kill()
picamera_process.process_starter.kill()
pitch_process.process_starter.kill()



/tmp/ipykernel_2702/1014377469.py:1: DeprecationWarning: old alias
  bt_ser_process_man.kill()
/tmp/ipykernel_2702/1014377469.py:2: DeprecationWarning: old alias
  bluetooth_control_process.process_starter.kill()
/tmp/ipykernel_2702/1014377469.py:3: DeprecationWarning: old alias
  logger_process.process_starter.kill()
/tmp/ipykernel_2702/1014377469.py:4: DeprecationWarning: old alias
  two_wheel_process.process_starter.kill()
/tmp/ipykernel_2702/1014377469.py:5: DeprecationWarning: old alias
  angular_speed_control_process.process_starter.kill()
/tmp/ipykernel_2702/1014377469.py:6: DeprecationWarning: old alias
  picamera_process.process_starter.kill()
/tmp/ipykernel_2702/1014377469.py:7: DeprecationWarning: old alias
  pitch_process.process_starter.kill()


In [8]:
from utils import mon_samples
mon_samples(pitch_process.outgoing_samples)

# playground

In [8]:
l = logger_starter.incoming_rpcs['get_logger'].call('TwoWheelsV3')()

In [10]:
import plotly.express as px
px.imshow(picamera_starter.outgoing_samples[0]())

KeyboardInterrupt: 